# Movie Recomendation System

### In this project we will be building a model which will recomend movies according to many parameters. We will be using Cosine Similarity and TF-IDF to complete the target

In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [174]:
df = pd.read_csv('movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [175]:
df.shape

(4803, 24)

In [176]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [177]:
df.nunique()

index                   4803
budget                   436
genres                  1168
homepage                1691
id                      4803
keywords                4219
original_language         37
original_title          4801
overview                4800
popularity              4802
production_companies    3697
production_countries     469
release_date            3280
revenue                 3297
runtime                  156
spoken_languages         544
status                     3
tagline                 3944
title                   4800
vote_average              71
vote_count              1609
cast                    4741
crew                    4776
director                2349
dtype: int64

In [178]:
# df_sel = ['genres', 'keywords', 'tagline', 'cast', 'director']
df_sel = ['genres', 'keywords', 'tagline', 'cast', 'director', 'original_language', 'overview', 'tagline']
df_sel

['genres',
 'keywords',
 'tagline',
 'cast',
 'director',
 'original_language',
 'overview',
 'tagline']

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [180]:
df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [181]:
df[df_sel].head()

,genres,keywords,tagline,cast,director,original_language,overview,tagline
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,en,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,en,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins."
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,en,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,en,Following the death of District Attorney Harve...,The Legend Ends
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,en,"John Carter is a war-weary, former military ca...","Lost in our world, found in another."


In [182]:
df[df_sel].isnull().sum()

genres                28
keywords             412
tagline              844
cast                  43
director              30
original_language      0
overview               3
tagline              844
dtype: int64

In [183]:
for i in df_sel:
    df[i] = df[i].fillna("")
df[df_sel].isnull().sum()

genres               0
keywords             0
tagline              0
cast                 0
director             0
original_language    0
overview             0
tagline              0
dtype: int64

In [184]:
# df_new = df['genres'] + '' + df['keywords'] + '' + df['tagline'] + '' + df['cast'] + '' + df['director']
df_new = df['genres'] + '' + df['keywords'] + '' + df['tagline'] + '' + df['cast'] + '' + df['director'] + '' + df['original_language'] + '' + df['overview']

In [185]:
df_new

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [187]:
vectorizer = TfidfVectorizer()

In [188]:
feature_vectors = vectorizer.fit_transform(df_new)

In [189]:
feature_vectors

<4803x44686 sparse matrix of type '<class 'numpy.float64'>'
	with 295804 stored elements in Compressed Sparse Row format>

In [190]:
feature_vectors.shape

(4803, 44686)

In [191]:
cos_sim = cosine_similarity(feature_vectors)
cos_sim

array([[1.        , 0.04634326, 0.02108158, ..., 0.02523981, 0.03220834,
        0.0067229 ],
       [0.04634326, 1.        , 0.03934628, ..., 0.04579739, 0.03190711,
        0.01639837],
       [0.02108158, 0.03934628, 1.        , ..., 0.02539329, 0.01199586,
        0.01472132],
       ...,
       [0.02523981, 0.04579739, 0.02539329, ..., 1.        , 0.03853042,
        0.02034653],
       [0.03220834, 0.03190711, 0.01199586, ..., 0.03853042, 1.        ,
        0.03414151],
       [0.0067229 , 0.01639837, 0.01472132, ..., 0.02034653, 0.03414151,
        1.        ]])

In [192]:
cos_sim.shape

(4803, 4803)

In [193]:
titles = df['title'].tolist()
titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [194]:
len(titles)

4803

In [195]:
movie_name = input(' Enter movie name : ')

 Enter movie name :  Avatar


In [196]:
import difflib
close_matches = difflib.get_close_matches(movie_name, titles)
close_matches

['Avatar']

In [197]:
one_match = close_matches[0]
one_match

'Avatar'

In [198]:
index_of_the_movie = df[df.title == one_match]['index'].values[0]
index_of_the_movie

0

In [199]:
similarity_score = list(enumerate(cos_sim[index_of_the_movie]))
similarity_score

[(0, 0.9999999999999999),
 (1, 0.046343263192810104),
 (2, 0.021081580816148675),
 (3, 0.018648634815661994),
 (4, 0.0739395126650764),
 (5, 0.031318245096298415),
 (6, 0.021317090146790993),
 (7, 0.05924704890301167),
 (8, 0.02218072479919755),
 (9, 0.02625065944109297),
 (10, 0.04007264841900668),
 (11, 0.029318081052504528),
 (12, 0.019581481093447296),
 (13, 0.023463514215909784),
 (14, 0.05386944574093818),
 (15, 0.027830281346943035),
 (16, 0.030027205662766596),
 (17, 0.026019304028478244),
 (18, 0.04475671029595688),
 (19, 0.031948617702448105),
 (20, 0.02550764206576438),
 (21, 0.01700013445202037),
 (22, 0.015103114059486257),
 (23, 0.030759611317351537),
 (24, 0.020546708250794573),
 (25, 0.024210589764749647),
 (26, 0.05403290163516432),
 (27, 0.07939222531276342),
 (28, 0.0204334864969967),
 (29, 0.04263097317022286),
 (30, 0.020491307402416163),
 (31, 0.0647166822528274),
 (32, 0.038432612582954405),
 (33, 0.024881759387937446),
 (34, 0.016743925623052443),
 (35, 0.033402

In [200]:
len(similarity_score)

4803

In [201]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
sorted_similar_movies

[(0, 0.9999999999999999),
 (1531, 0.14437139678653407),
 (3158, 0.14281129605222478),
 (369, 0.12671850559171088),
 (942, 0.11874913786089852),
 (94, 0.11763587978003684),
 (1914, 0.11620626722952893),
 (3604, 0.11583919588730432),
 (305, 0.11552636522225128),
 (432, 0.11400517386383191),
 (1275, 0.11170593366157423),
 (1759, 0.10730774984152529),
 (56, 0.10621596598800978),
 (529, 0.10572875203470017),
 (549, 0.1052651175838697),
 (461, 0.10337087828075815),
 (43, 0.10290908767260759),
 (1951, 0.10238675282437247),
 (150, 0.10192201275331887),
 (838, 0.09757133989919006),
 (1341, 0.09633438899358648),
 (1213, 0.09616853635216918),
 (1354, 0.09585296970260657),
 (239, 0.09298633526862596),
 (3730, 0.0922776063723108),
 (47, 0.09193018440619648),
 (775, 0.09104826830357698),
 (2403, 0.09036662679498701),
 (3668, 0.08987801293837264),
 (3105, 0.08985565633226378),
 (1217, 0.0892958546830366),
 (111, 0.0892344480290116),
 (634, 0.08857000273585686),
 (300, 0.08768101284616094),
 (3439, 0.

In [202]:
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


Movies suggested for you : 

1 . Avatar
2 . Moonraker
3 . Alien
4 . Lara Croft Tomb Raider: The Cradle of Life
5 . The Book of Life
6 . Guardians of the Galaxy
7 . Lifeforce
8 . Apollo 18
9 . Treasure Planet
10 . Deep Impact
11 . Sunshine
12 . The Right Stuff
13 . Star Trek Beyond
14 . Tears of the Sun
15 . Sphere
16 . Lost in Space
17 . Terminator Salvation
18 . Space Dogs
19 . Men in Black II
20 . Alien³
21 . The Inhabited Island
22 . Aliens vs Predator: Requiem
23 . Space Chimps
24 . Gravity
25 . Cargo
26 . Star Trek Into Darkness
27 . Supernova
28 . Aliens
29 . Capricorn One
